In [2]:
from importlib.metadata import version

#import tiktoken
import torch

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

ModuleNotFoundError: No module named 'torch'

In [2]:
from importlib.metadata import version